In [320]:
import re
import sys
import math
import time
import json
import csv
import pickle
import random
import requests
import pprint
import numpy as np
import pandas as pd
import datetime as dt
import string
from itertools import combinations
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix
from langdetect import detect
from datetime import datetime
from sklearn import metrics
import scipy.stats as stat
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.dates as mdates
from scipy.stats import chisquare, ttest_ind
from matplotlib.dates import DateFormatter
from collections import defaultdict, Counter

In [321]:
pd.set_option('mode.chained_assignment', None)
pd.set_option('display.max_columns', None)

In [322]:
data_root = '/home/haopeng/Data/Retraction/'
altmetric_start_date = datetime.strptime('2011-06-10', '%Y-%m-%d')
altmetric_end_date = datetime.strptime('2019-10-08', '%Y-%m-%d')
CIs = {'90': 1.645, '95': 1.96, '99': 2.576}
day = 24*3600

In [323]:
blog_df = pd.read_csv(data_root + 'blog_mentions_correction.csv', header = 0)
blog_set = set(blog_df.loc[~blog_df.maybe_news.isnull(), 'blog_name'])

def correct_blogs(data, error_blogs):
    for xid, paper in data.items():
        if 'blogs' in paper['posts']:
            blogs_li = paper['posts']['blogs']
            news_li = []
            if 'news' in paper['posts']:
                news_li = paper['posts']['news']
            blogs_li_new = []
            for item in blogs_li:
                try:
                    if item['author']['name'] in error_blogs:
                        news_li.append(item)
                    else:
                        blogs_li_new.append(item)
                except:
                    pass
            paper['posts']['news'] = news_li
            paper['posts']['blogs'] = blogs_li_new
    return data

### Load Alt data

In [324]:
doi_alt_data = {}

with open(data_root+"retraction_altmetric.json", 'r') as ffile:
    for row in ffile:
        record = json.loads(row)
        doi = record['citation']['doi'].lower()
        doi_alt_data[doi] = record

doi_alt_data = correct_blogs(doi_alt_data, blog_set)

In [325]:
len(doi_alt_data)

6838

In [326]:
# many retracted papers have missing values on author/affil rank, thus no matches.
doi_control_aids ={}

with open(data_root+"revision/doi_control_alt_ids_revision_v6.json", 'r') as ffile:
    for line in ffile:
        line = json.loads(line)
        doi_control_aids[line['doi']] = [str(x) for x in line['alt_ids']]

In [327]:
len(doi_control_aids)

3851

In [328]:
with open(data_root+'df_retract_clean.pickle', 'rb') as file:
    retract_df = pickle.load(file)

In [329]:
len(retract_df)

6680

### Exclude papers before Alt launch date

In [330]:
retract_df = retract_df.loc[retract_df.pub_date_corrected.apply(lambda dttt: (dttt-altmetric_start_date).total_seconds() > 0)]
retract_df.index = range(len(retract_df))

In [331]:
len(retract_df)

4167

In [332]:
retract_df = retract_df.loc[retract_df.OriginalPaperDOI.isin(doi_control_aids)]
retract_df.index = range(len(retract_df))

In [333]:
len(retract_df)

3659

In [334]:
doi_ret_date = dict(zip(retract_df['OriginalPaperDOI'], retract_df['RetractionDate']))

In [335]:
altid_data = {}

with open(data_root+"revision/control_alt_papers_revision_v6.json", 'r') as ffile:
    for row in ffile:
        paper = json.loads(row)
        altid = str(paper['altmetric_id'])
        altid_data[altid] = paper
        
altid_data = correct_blogs(altid_data, blog_set)

In [336]:
control_aid_doi = {}

for doi in retract_df.OriginalPaperDOI:
    for aid in doi_control_aids[doi]:
        control_aid_doi[aid] = doi

In [337]:
len(control_aid_doi)

18295

### Load tweets

In [338]:
with open(data_root + 'tweets_processed.pickle', 'rb') as ofile:
    tid_date, tid_text, tid_author, tid_follow_cn, retweet_org_ids = pickle.load(ofile)

In [339]:
len(tid_date)

159408

In [340]:
len(tid_text)

159408

In [341]:
len(tid_follow_cn)

159408

In [342]:
len(retweet_org_ids)

87670

### Load tweets for new control papers 

In [343]:
tids_add = set()

for aid in altid_data:
    paper = altid_data[aid]
    if 'posts' in paper and 'twitter' in paper['posts']:
        for item in paper['posts']['twitter']:
            tid = item['tweet_id']
            if tid not in tid_date:
                tids_add.add(tid)

In [344]:
len(tids_add)

106798

In [265]:
# get tweets from the full collection.

tid_date_new = {}
tid_text_new = {}
tid_follow_cn_new = {}
retweet_org_ids_new = {}

with open(data_root+'second_match/tweets_clean.json', 'r') as ifile:
    for line in ifile:
        line = json.loads(line)
        tid, tdate = line['id_str'], line['created_at']
        if tid in tids_add:
            re_status = False
            text = ""
            try:
                text = line['retweeted_status']['text']
                org_tid = line['retweeted_status']['id_str']
                re_status = True
            except:  # Not a Retweet
                text = line['text']
            try:
                if detect(text) == 'en':
                    tid_date_new[tid] = datetime.strptime(tdate, '%a %b %d %H:%M:%S +0000 %Y')
                    tid_text_new[tid] = text
                    tid_follow_cn_new[tid] = int(line['user']['followers_count'])
                    if re_status:
                        retweet_org_ids_new[tid] = org_tid
            except:
                pass

In [266]:
len(tid_date_new)

75393

In [267]:
len(tid_text_new)

75393

In [268]:
len(retweet_org_ids_new)

43175

In [269]:
# with open(data_root + 'revision/tweets_processed_revision_v6.pickle', 'wb') as ofile:
#     pickle.dump([tid_date_new, tid_text_new, tid_follow_cn_new, retweet_org_ids_new], ofile)

In [345]:
with open(data_root + 'revision/tweets_processed_revision_v6.pickle', 'rb') as ofile:
    tid_date_new, tid_text_new, tid_follow_cn_new, retweet_org_ids_new = pickle.load(ofile)

In [346]:
for tid, val in tid_date_new.items():
    tid_date[tid] = val
    
for tid, val in tid_text_new.items():
    tid_text[tid] = val
    
for tid, val in tid_follow_cn_new.items():
    tid_follow_cn[tid] = val

for tid, val in retweet_org_ids_new.items():
    retweet_org_ids[tid] = val

In [347]:
del tid_date_new, tid_text_new, tid_follow_cn_new, retweet_org_ids_new

In [348]:
len(tid_date)

243490

### Get tweets prior retraction (after correcting date)

In [349]:
cand_tids_ret = dict()
cand_tids_control = dict()

for doi, ret_date in doi_ret_date.items():
    paper = doi_alt_data[doi]
    cand_tids_ret[doi] = []
    if 'posts' in paper and 'twitter' in paper['posts']:
        for item in paper['posts']['twitter']:
            tid = item['tweet_id']
            # exclude retweets
            if tid in tid_date and tid not in retweet_org_ids:
                dttt = tid_date[tid]
                if (ret_date - dttt).total_seconds() > 0:
                    cand_tids_ret[doi].append(tid)
                    
for altid, matched_doi in control_aid_doi.items():
    ret_date = doi_ret_date[matched_doi]
    paper = altid_data[altid]
    cand_tids_control[altid] = []
    if 'posts' in paper and 'twitter' in paper['posts']:
        for item in paper['posts']['twitter']:
            tid = item['tweet_id']
            if tid in tid_date and tid not in retweet_org_ids:
                dttt = tid_date[tid]
                if (ret_date - dttt).total_seconds() > 0:
                    cand_tids_control[altid].append(tid)

In [350]:
len(cand_tids_ret)

3659

In [351]:
len(cand_tids_control)

18295

In [352]:
sum([len(v) for v in cand_tids_ret.values()])

14747

In [353]:
sum([len(v) for v in cand_tids_control.values()])

28042

### Label uncertain tweets

In [354]:
word_li = set()
with open(data_root+'doubt_words.txt', 'r') as ifile:
    for line in ifile:
        w = line.strip()
        if w not in word_li:
            word_li.add(w)
        else:
            print(w)
            
# distrust, disbelief, doubt, concern, question, confusion, suspicion, credibility, uncertain, unsure, criticism
def is_doubt(text):
    text = text.lower()
    for w in word_li:
        if w in text:
            return 1
    return 0

In [355]:
len(word_li)

76

Manually label control tweets that meet the heuristic

In [356]:
cand_tids = set()

for doi, tids in cand_tids_ret.items():
    for tid in tids:
        if is_doubt(tid_text[tid]):
            cand_tids.add(tid)

In [357]:
len(cand_tids)

3388

In [358]:
cand_tids = set()

for altid, tids in cand_tids_control.items():
    for tid in tids:
        if is_doubt(tid_text[tid]):
            cand_tids.add(tid)

In [359]:
len(cand_tids)

3870

Reuse already labelled ones

In [550]:
final_df_old = pd.read_csv(data_root+'cand_uncertain_tweets_mvotes.csv', quoting=csv.QUOTE_ALL, header=0, dtype={'tid': 'str'})


In [551]:
tids_done = set(final_df_old.tid.tolist())

In [552]:
len(tids_done)

9789

In [363]:
cand_tids = [tid for tid in cand_tids if tid not in tids_done]

In [364]:
len(cand_tids)

3782

In [315]:
cand_tweets_df = pd.DataFrame(cand_tids, columns=['tid'])
cand_tweets_df['text'] = cand_tweets_df.tid.map(tid_text)
cand_tweets_df['is_uncertain'] = ''
cand_tweets_df['note'] = ''

In [316]:
signal_words = []
for text in cand_tweets_df.text:
    text = text.lower()
    for w in word_li:
        if w in text:
            signal_words.append(w)
            break
    else:
        signal_words.append('')

cand_tweets_df['signal_word'] = signal_words

In [317]:
cand_tweets_df.head(10)

,tid,text,is_uncertain,note,signal_word
0,1064912008590606338,Tips on what to eat as munchies when you are r...,,,?
1,924192112077185024,I agree @SenAlexander Harming persons with com...,,,problem
2,735866904632397824,Nature ハイライト：ギャップ結合形成阻害剤による転移の減少 | Nature | Na...,,,publish
3,583761754079002625,One for you @FfitzP! @JAMAPeds: Why aren’t the...,,,?
4,777521309290991620,#CFS If HHV-6 really is a form of ASFV will it...,,,problem
5,478508111456509953,Are trial participants adequately safeguarded ...,,,?
6,159340156703354882,Physical activity is positively related to aca...,,,academic
7,661589514478084096,"Does ""The Talk"" matter? New review of research...",,,?
8,1063456232399355904,"Nevertheless, #USPSTF recently published in @J...",,,publish
9,634259593338580992,HLA Typing from 1000 Genomes Whole Genome and ...,,,data


In [318]:
cand_tweets_df.to_csv(data_root+'revision/cand_uncertain_tweets_revision.csv', quoting=csv.QUOTE_ALL, header=True, index=False)

Agreement (all tweets)

In [367]:
hao_df = pd.read_csv(data_root+'revision/cand_uncertain_tweets_hao.csv', quoting=csv.QUOTE_ALL, dtype={'tid': str})

In [368]:
len(hao_df)

3782

In [440]:
rs = {}
names = ['hao', 'henry', 'rod']
for name in names:
    r1 = pd.read_csv(data_root+'revision/cand_uncertain_tweets_%s.csv'%name, quoting=csv.QUOTE_ALL, dtype={'tid': str})
    r1 = r1[['tid', 'text', 'is_uncertain']]
    r1.loc[r1.text.apply(lambda x: 'True or false?' in x), 'is_uncertain'] = 1
    r1 = r1.replace({np.NaN: 0})    
    r1 = r1.astype({'is_uncertain': 'int64'})
    r1 = r1.sort_values('tid')
    r1.index = range(len(r1))
    rs[name] = r1

In [441]:
ks = []
for n1, n2 in combinations(names, r=2):
    r1 = rs[n1]
    r2 = rs[n2]
    kappa = cohen_kappa_score(r1['is_uncertain'], r2['is_uncertain'])
    ks.append(kappa)
    print(n1, n2, kappa)

hao henry 0.7940209332046353
hao rod 0.8429062180768174
henry rod 0.7047505194278723


In [442]:
np.sum(rs['henry']['is_uncertain'])

253

In [443]:
np.sum(rs['rod']['is_uncertain'])

357

In [444]:
np.sum(rs['hao']['is_uncertain'])

274

In [445]:
confusion_matrix(rs['hao']['is_uncertain'], rs['henry']['is_uncertain'])

array([[3468,   40],
       [  61,  213]])

In [446]:
confusion_matrix(rs['rod']['is_uncertain'], rs['henry']['is_uncertain'])

array([[3394,   31],
       [ 135,  222]])

In [447]:
confusion_matrix(rs['hao']['is_uncertain'], rs['rod']['is_uncertain'])

array([[3421,   87],
       [   4,  270]])

In [448]:
votes = np.array([rs[name]['is_uncertain'].values for name in names]).T
votes = np.sum(votes, axis=1)

In [449]:
np.unique(votes, return_counts=True)

(array([0, 1, 2, 3]), array([3391,  110,   69,  212]))

In [450]:
tid_votes = dict(zip(rs['hao'].tid, votes))

In [451]:
len(tid_votes)

3782

In [452]:
bad_ix = (votes == 1) | (votes == 2)

In [453]:
bad_df = rs['hao'].loc[bad_ix, ['tid', 'text']]
bad_df = bad_df.sample(frac = 1.0, random_state=10)
bad_df.index = range(len(bad_df))

In [454]:
len(bad_df)

179

In [456]:
bad_df.to_csv(data_root+'revision/cand_uncertain_tweets_disagree.csv', quoting=csv.QUOTE_ALL, header=True, index=False)

In [496]:
agnes = pd.read_csv(data_root+'revision/cand_uncertain_tweets_disagree_agnes.csv', quoting=csv.QUOTE_ALL, dtype={'tid': str})

In [497]:
len(agnes)

90

In [498]:
agnes['votes'] = agnes.tid.map(tid_votes)

In [499]:
agnes['new_votes'] = agnes.apply(lambda row: row['is_uncertain'] + row['votes'], axis = 1)

In [500]:
agnes.loc[agnes.new_votes == 2]

,tid,text,is_uncertain,Unnamed: 3,votes,new_votes
2,461672468369776641,""" http://t.co/H9CqDQ7SHL"" Why, why, why is th...",0,NaN,2,2
3,729975825143980033,Note of the methodological flaws in the paper ...,1,NaN,1,2
7,1049940115349524480,@iskander do you know wether Epi-Seq (https://...,0,NaN,2,2
8,824094729444855808,I cannot wait to hear with others think of thi...,0,NaN,2,2
10,339210198734475264,Conservative management of symptomatic knee os...,1,NaN,1,2
11,1093725516186689537,"So there are millions, not trillions of *OTUs*...",0,NaN,2,2
12,518033505121755137,Poor sleep quality is associated with #suicide...,0,NaN,2,2
15,911508540665278464,#Retraction @CambridgeUP: COI &amp; hidden com...,1,This shouldn't have contained the retraction f...,1,2
16,365143014668115969,New JAMA study says combat deployments NOT lin...,1,NaN,1,2
18,1092067869347721216,65% of US pregnant women take #TYLENOL \n\nIS ...,0,NaN,2,2


In [501]:
agnes.new_votes.value_counts()

1    37
2    35
3    18
Name: new_votes, dtype: int64

In [502]:
tid_mvotes = tid_votes.copy()

In [503]:
# treat three 1s as positive case
for tid, mvote in tid_mvotes.items():
    if mvote == 3:
        tid_mvotes[tid] = 1

In [504]:
# deal with one or two 1s, depending on their new total vote (four labels)
for tid, nvote in zip(agnes.tid, agnes.new_votes):
    if nvote == 1:
        tid_mvotes[tid] = 0
    elif nvote == 3:
        tid_mvotes[tid] = 1
    else:
        del tid_mvotes[tid]

In [505]:
daniel = pd.read_csv(data_root+'revision/cand_uncertain_tweets_disagree_daniel.csv', quoting=csv.QUOTE_ALL, dtype={'tid': str})

In [506]:
len(daniel)

89

In [507]:
daniel['votes'] = daniel.tid.map(tid_votes)

In [508]:
daniel['new_votes'] = daniel.apply(lambda row: row['is_uncertain'] + row['votes'], axis = 1)

In [509]:
daniel.loc[daniel.new_votes == 2]

,tid,text,is_uncertain,votes,new_votes
0,228618947816218624,"""Think before you stink."" Seriously. http://t....",0,2,2
7,264051406586126338,http://t.co/QcuF95tU\n\nDopamine receptor anta...,0,2,2
9,864448422983737344,Nature biotech will not retract this obviously...,1,1,2
10,953758969000026114,"What if in-person doctor visits were the 2nd, ...",0,2,2
11,750683971625771008,Issues cause your antibodies not specific enou...,0,2,2
15,1108890701742264321,You're right in that the CABANA Trial is an in...,0,2,2
16,534640895157870593,@sarabran @Lilydunnwriter an old idea + contro...,1,1,2
17,1041764078794858496,"This is lie. Alcohol is the problem.\n\n""3 yrs...",1,1,2
18,647392128474021888,Median follow-up was only 9 months. Too shor...,1,1,2
19,697637442023727104,“Editorial Introduction to 'opportunity Makes ...,0,2,2


In [510]:
daniel.new_votes.value_counts()

1    44
2    32
3    13
Name: new_votes, dtype: int64

In [511]:
# deal with one or two 1s, depending on their new total vote (four labels)
for tid, nvote in zip(daniel.tid, daniel.new_votes):
    if nvote == 1:
        tid_mvotes[tid] = 0
    elif nvote == 3:
        tid_mvotes[tid] = 1
    else:
        del tid_mvotes[tid]

In [536]:
len(tid_mvotes)

3715

In [543]:
final_df = rs['hao'].copy()

In [544]:
final_df = final_df.drop(columns=['is_uncertain'])

In [539]:
# treat (0, 0, 1, 1) cases as negative
final_df['majority_vote'] = final_df.tid.apply(lambda tid: tid_mvotes[tid] if tid in tid_mvotes else 0)

In [546]:
len(final_df)

3782

In [547]:
final_df.head()

,tid,text,majority_vote
0,1000406217133608960,Results from National Cancer Database in men w...,0
1,1000876218974171137,"Presentation Pet Peeves\n""As an academic gener...",0
2,1000890492501090304,I had the “Neurology stable: what does that me...,0
3,1001047682025127936,"I love the testiness in this article, e.g.:\n\...",1
4,1001070522560667650,Vibrational and electrical properties of Cu2−x...,0


In [542]:
final_df.majority_vote.value_counts()

0    3539
1     243
Name: majority_vote, dtype: int64

In [519]:
final_df.to_csv(data_root+'revision/cand_uncertain_tweets_revision_mvotes.csv', quoting=csv.QUOTE_ALL, header=True, index=False)


In [545]:
# treat (0, 0, 1, 1) cases as positive
final_df['majority_vote'] = final_df.tid.apply(lambda tid: tid_mvotes[tid] if tid in tid_mvotes else 1)

In [548]:
final_df.majority_vote.value_counts()

0    3472
1     310
Name: majority_vote, dtype: int64

In [549]:
final_df.to_csv(data_root+'revision/cand_uncertain_tweets_revision_mvotes_bar.csv', quoting=csv.QUOTE_ALL, header=True, index=False)


Prepare data for publishing

In [520]:
final_df = pd.read_csv(data_root+'revision/cand_uncertain_tweets_revision_mvotes.csv', quoting=csv.QUOTE_ALL, header=0, dtype={'tid': 'str'})


In [521]:
final_df['meet_heuristic'] = 1

In [522]:
votes = np.array([rs[name]['is_uncertain'].values for name in names]).T

In [523]:
tid_votes = dict(zip(rs['hao'].tid, votes))

In [524]:
final_df['three_labels'] = final_df.tid.map(tid_votes)

In [525]:
tem_df = agnes.append(daniel)
tid_fourth = dict(zip(tem_df['tid'], tem_df['is_uncertain']))

In [526]:
final_df['fourth_label'] = final_df.tid.map(tid_fourth)

In [527]:
final_df.head()

,tid,text,majority_vote,meet_heuristic,three_labels,fourth_label
0,1000406217133608960,Results from National Cancer Database in men w...,0,1,"[0, 0, 0]",NaN
1,1000876218974171137,"Presentation Pet Peeves\n""As an academic gener...",0,1,"[0, 0, 0]",NaN
2,1000890492501090304,I had the “Neurology stable: what does that me...,0,1,"[0, 0, 0]",NaN
3,1001047682025127936,"I love the testiness in this article, e.g.:\n\...",0,1,"[0, 0, 1]",1.0
4,1001070522560667650,Vibrational and electrical properties of Cu2−x...,0,1,"[0, 0, 0]",NaN


In [528]:
filter_df = []

for altid, tids in cand_tids_control.items():
    for tid in tids:
        if not is_doubt(tid_text[tid]):
            filter_df.append(tid)

In [529]:
filter_df = pd.DataFrame(filter_df, columns=['tid'])

In [530]:
filter_df['text'] = filter_df.tid.map(tid_text)

In [531]:
filter_df['meet_heuristic'] = 0

In [532]:
len(filter_df)

24172

In [533]:
all_df = final_df.append(filter_df)

In [534]:
all_df.head(5)

,tid,text,majority_vote,meet_heuristic,three_labels,fourth_label
0,1000406217133608960,Results from National Cancer Database in men w...,0.0,1,"[0, 0, 0]",NaN
1,1000876218974171137,"Presentation Pet Peeves\n""As an academic gener...",0.0,1,"[0, 0, 0]",NaN
2,1000890492501090304,I had the “Neurology stable: what does that me...,0.0,1,"[0, 0, 0]",NaN
3,1001047682025127936,"I love the testiness in this article, e.g.:\n\...",0.0,1,"[0, 0, 1]",1.0
4,1001070522560667650,Vibrational and electrical properties of Cu2−x...,0.0,1,"[0, 0, 0]",NaN


In [554]:
len(all_df)

27954

In [535]:
all_df.to_csv(data_root+'revision/tweets_uncertain_label_revision.csv', quoting=csv.QUOTE_ALL, header=True, index=False, \
             columns=['tid', 'text', 'meet_heuristic', 'three_labels', 'fourth_label', 'majority_vote'])

In [643]:
all_df = pd.read_csv(data_root+'revision/tweets_uncertain_label_revision.csv', quoting=csv.QUOTE_ALL, header=0, dtype={'tid': 'str'})


Combine previous data (tweets for retracted paper in the initial version) with new data (tweets for newly matched control papers in the revision)

In [644]:
share_df_old = pd.read_csv(data_root+'tweets_uncertain_label.csv', quoting=csv.QUOTE_ALL, header=0, dtype={'tid': 'str'})


In [645]:
len(share_df_old)

50158

In [646]:
tem_all = all_df.append(share_df_old)

In [647]:
tem_all.head()

,tid,text,meet_heuristic,three_labels,fourth_label,majority_vote
0,1000406217133608960,Results from National Cancer Database in men w...,1,[0 0 0],NaN,0.0
1,1000876218974171137,"Presentation Pet Peeves\n""As an academic gener...",1,[0 0 0],NaN,0.0
2,1000890492501090304,I had the “Neurology stable: what does that me...,1,[0 0 0],NaN,0.0
3,1001047682025127936,"I love the testiness in this article, e.g.:\n\...",1,[0 0 1],1.0,0.0
4,1001070522560667650,Vibrational and electrical properties of Cu2−x...,1,[0 0 0],NaN,0.0


In [648]:
len(tem_all)

78112

In [649]:
tem_all = tem_all.drop_duplicates(subset=['tid'])
tem_all.index = range(len(tem_all))

In [650]:
len(tem_all)

77514

In [651]:
all_tids = set()

for doi, tids in cand_tids_ret.items():
    for tid in tids:
        all_tids.add(tid)
        
for altid, tids in cand_tids_control.items():
    for tid in tids:
        all_tids.add(tid)

In [652]:
len(all_tids)

42767

In [653]:
tem_all = tem_all.loc[tem_all.tid.isin(all_tids)]
tem_all.index = range(len(tem_all))

In [654]:
tem_all.head()

,tid,text,meet_heuristic,three_labels,fourth_label,majority_vote
0,1000406217133608960,Results from National Cancer Database in men w...,1,[0 0 0],NaN,0.0
1,1000876218974171137,"Presentation Pet Peeves\n""As an academic gener...",1,[0 0 0],NaN,0.0
2,1000890492501090304,I had the “Neurology stable: what does that me...,1,[0 0 0],NaN,0.0
3,1001047682025127936,"I love the testiness in this article, e.g.:\n\...",1,[0 0 1],1.0,0.0
4,1001070522560667650,Vibrational and electrical properties of Cu2−x...,1,[0 0 0],NaN,0.0


In [655]:
len(tem_all)

42766

In [656]:
tem_all.dtypes

tid                object
text               object
meet_heuristic      int64
three_labels       object
fourth_label      float64
majority_vote     float64
dtype: object

In [657]:
cand_label_df = tem_all.loc[tem_all.meet_heuristic==1]

In [658]:
len(cand_label_df)

7036

In [659]:
len(cand_label_df.loc[cand_label_df.fourth_label.isna()])

6201

In [660]:
cand_label_df.loc[cand_label_df.fourth_label.isna()]['majority_vote'].value_counts()

0.0    5481
1.0     720
Name: majority_vote, dtype: int64

In [661]:
dis_df = cand_label_df.loc[~cand_label_df.fourth_label.isna()]

In [662]:
len(dis_df)

835

In [670]:
# num of disambiguous ones (2 vs 2)

cn = 0
for lab, fourth in zip(dis_df['three_labels'], dis_df['fourth_label']):
    if '.' in lab:
        lab_li = [int(x) for x in lab[1:-2].split('. ')]
    else:
        lab_li = [int(x) for x in lab[1:-1].split(' ')]
    if np.sum(lab_li) + fourth == 2:
        cn += 1

In [671]:
cn

359

In [672]:
# agreement for tweets labeled in the revision

len(cand_label_df.loc[cand_label_df.tid.isin(final_df.tid)])

# hao henry 0.7940209332046353
# hao rod 0.8429062180768174
# henry rod 0.7047505194278723

# avg: 0.78

3782

In [673]:
# calculate agreement for tweets labeled in the initial submission

cand_label_df_ini = cand_label_df.loc[~cand_label_df.tid.isin(final_df.tid)]

In [675]:
len(cand_label_df_ini)

3254

In [676]:
three_labs = []
for lab in cand_label_df_ini.three_labels.tolist():
    three_labs.append([int(x) for x in lab[1:-2].split('. ')])

In [677]:
three_labs = np.array(three_labs)

In [678]:
three_labs.shape

(3254, 3)

In [679]:
lii = []
for x, y in combinations([0, 1, 2], r=2):
    agg = cohen_kappa_score(three_labs[:, x], three_labs[:, y])
    lii.append(agg)
    print(agg)

0.7031831868019676
0.6271233428256657
0.6018313956009123


In [680]:
np.mean(lii)

0.6440459750761819

In [681]:
# weighted average agreement
0.644 * len(cand_label_df_ini)/len(cand_label_df) + 0.78 * (1 - len(cand_label_df_ini)/len(cand_label_df))

0.7171028993746447

In [682]:
tem_all.to_csv(data_root+'revision/all_tweets_uncertain_label_revision_share.csv', quoting=csv.QUOTE_ALL, header=True, index=False, \
             columns=['tid', 'meet_heuristic', 'three_labels', 'fourth_label', 'majority_vote'])